In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
filename = input('Enter file name')
with open(filename+'.txt') as f:
    shakespeare_text = f.read()
print("Total character count : ", len(shakespeare_text))
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

Enter file nameshakespeare_sonnets
Total character count :  94275


In [3]:
word_count = tokenizer.word_counts
document_count = tokenizer.document_count
word_index = tokenizer.word_index

print("Token count : ", word_count, "\n")
print("Document size : ", document_count, "\n")
print("Index of tokens", word_index, "\n")
print("Unique characters : ", len(word_count), "\n")

Token count :  OrderedDict([('t', 7216), ('h', 5070), ('e', 9249), (' ', 15679), ('s', 4991), ('o', 5707), ('n', 4521), ('\n', 2468), ('b', 1230), ('y', 1986), ('w', 1898), ('i', 4677), ('l', 3093), ('a', 4941), ('m', 2095), ('k', 553), ('p', 1011), ('r', 4183), ('f', 1663), ('c', 1342), ('u', 2320), ('d', 2763), (',', 1880), ("'", 386), ('g', 1358), ('v', 925), (':', 160), ('-', 83), ('.', 396), ('z', 20), (';', 31), ('x', 60), ('?', 92), ('q', 51), ('j', 68), ('(', 43), (')', 43), ('!', 23)]) 

Document size :  1 

Index of tokens {' ': 1, 'e': 2, 't': 3, 'o': 4, 'h': 5, 's': 6, 'a': 7, 'i': 8, 'n': 9, 'r': 10, 'l': 11, 'd': 12, '\n': 13, 'u': 14, 'm': 15, 'y': 16, 'w': 17, ',': 18, 'f': 19, 'g': 20, 'c': 21, 'b': 22, 'p': 23, 'v': 24, 'k': 25, '.': 26, "'": 27, ':': 28, '?': 29, '-': 30, 'j': 31, 'x': 32, 'q': 33, '(': 34, ')': 35, ';': 36, '!': 37, 'z': 38} 

Unique characters :  38 



In [4]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
print(encoded.shape)

(94275,)


In [5]:
train_size = encoded.shape[0]
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
n_steps = 100
window_length = n_steps+1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [6]:
batch_size = 128
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth = len(word_count)), Y_batch)
)
dataset = dataset.prefetch(1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [7]:
import os
import time

model_name = int(time.time())

root_logdir = os.path.join(os.curdir, f"logs_{filename}")
run_id = time.strftime("date_%Y_%m_%d-time_%H_%M_%S")
log_dir = os.path.join(root_logdir, str(model_name)+"\\"+run_id)
print(log_dir)

root_filepath = os.path.join(os.curdir, f"checkpoints_{filename}")
filepath = os.path.join(root_filepath, str(model_name)+f"\\{model_name}.h5")
os.mkdir(os.path.join(root_filepath, str(model_name)))
print(filepath)

.\logs_shakespeare_sonnets\1624468336\date_2021_06_23-time_22_42_16
.\checkpoints_shakespeare_sonnets\1624468336\1624468336.h5


In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath, monitor='loss', verbose=1, save_best_only=False
    ),
    keras.callbacks.TensorBoard(
        log_dir=log_dir, histogram_freq=1,
        update_freq=10, write_graph=True
    )
]

In [9]:
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(None, len(word_count))),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(len(word_count), activation='softmax'))
])
model.summary()
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(
    dataset, epochs=1, batch_size=batch_size,
    callbacks=callbacks
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, None, 128)         64512     
_________________________________________________________________
gru_1 (GRU)                  (None, None, 64)          37248     
_________________________________________________________________
gru_2 (GRU)                  (None, None, 64)          24960     
_________________________________________________________________
time_distributed (TimeDistri (None, None, 38)          2470      
Total params: 129,190
Trainable params: 129,190
Non-trainable params: 0
_________________________________________________________________
      1/Unknown - 0s 0s/step - loss: 3.6366 - accuracy: 0.0217WARNING:tensorflow:From C:\Users\souna\anaconda3\envs\ml\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and wi